In [1]:
import pandas as pd
import numpy as np
import keras
import matplotlib.image as mpimg
import cv2
import pickle
from keras.models import model_from_json
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [23]:
# Load the images into pandas dataframe
df = pd.read_csv("driving_log.csv", header=None, skipinitialspace=True, names=["center", "left", "right",  "angle", "throttle", "brake", "speed"])

# center+left+right
center = pd.DataFrame({"image_path": df["center"],
                       "angle": df["angle"]
             })
left = pd.DataFrame({"image_path": df["left"],
                       "angle": df["angle"]+0.3
             })
right = pd.DataFrame({"image_path": df["right"],
                       "angle": df["angle"]-0.3
             })

frame = [center, left, right]
df = pd.concat(frame, ignore_index=True)


X = df["image_path"]
Y = df["angle"]
Y = Y.astype(np.float32)

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.1, random_state=1)


In [24]:
X_train.reindex()
X_train.iloc[0]
X_train.iloc[0]

'/Users/yifei/Documents/P3_new/IMG/right_2016_12_11_20_35_33_474.jpg'

In [25]:
img_rows, img_cols = 66, 200


def image_preprocess(path):
    image = mpimg.imread(path, format='jpg')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    shape = image.shape
    image = image[int(shape[0]/3):shape[0], 0:shape[1]]
    image = cv2.resize(image, (img_cols, img_rows), interpolation=cv2.INTER_AREA)
    image = np.array(image).astype("float32")
    return image

def batchgen(X, Y):
    while 1:
        for i in range(len(X)):
            image = image_preprocess(X.iloc[i])
            image = image.reshape([1, img_rows, img_cols, 1])
            y = np.array([[Y.iloc[i]]])
            yield image, y

# datagen = ImageDataGenerator(
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True)

In [26]:
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

(4409,)
(4409,)
(490,)
(490,)


In [27]:
# Build the network

from keras.models import Sequential
from keras.layers import Dense, Input, Activation, Dropout, MaxPooling2D, Convolution2D, Lambda, ELU, Flatten
from keras.utils import np_utils
from keras.optimizers import Adam

            
 
input_shape = (66, 200, 1)
pool_size = (2, 3)
img_rows, img_cols = 66, 200

batch_size = 5
samples_per_epoch = int(len(X_train)/batch_size)
nb_epoch = 200
val_size = int(samples_per_epoch/10.0)

model = Sequential()
# model.add(MaxPooling2D(pool_size=pool_size,input_shape=input_shape))
model.add(Lambda(lambda x: x/127.5 - 1., input_shape=input_shape))
model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same"))
model.add(ELU())
model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same"))
model.add(ELU())
model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same"))
model.add(Flatten())
model.add(Dropout(.2))
model.add(ELU())
model.add(Dense(512))
model.add(Dropout(.5))
model.add(ELU())
model.add(Dense(1))
model.compile(optimizer="adam", loss="mse")
model.summary()


history = model.fit_generator(batchgen(X_train, Y_train),
                    samples_per_epoch=samples_per_epoch, 
                    nb_epoch=nb_epoch,
                    validation_data=batchgen(X_val, Y_val),
                    nb_val_samples=val_size,
                    verbose=1)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_5 (Lambda)                (None, 66, 200, 1)    0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
convolution2d_13 (Convolution2D) (None, 17, 50, 16)    1040        lambda_5[0][0]                   
____________________________________________________________________________________________________
elu_17 (ELU)                     (None, 17, 50, 16)    0           convolution2d_13[0][0]           
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 9, 25, 32)     12832       elu_17[0][0]                     
___________________________________________________________________________________________

KeyboardInterrupt: 

In [22]:
# load model to .json
import json
json_string = model_json = model.to_json()
with open('model.json', 'w') as f:
    json.dump(model_json, f)



# save weights as .h5
json_string = model.to_json()
model = model_from_json(json_string)
model.save_weights("model.h5")